In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [ ]:
df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [ ]:
df = df.drop(columns='id',axis=1)

In [ ]:
df

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,1166,D-penicillamine,16839,F,N,N,N,N,0.8,309.0,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,C
7901,1492,Placebo,17031,F,N,Y,N,N,0.9,260.0,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,C
7902,1576,D-penicillamine,25873,F,N,N,Y,S,2.0,225.0,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,D
7903,3584,D-penicillamine,22960,M,N,Y,N,N,0.7,248.0,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,D


In [ ]:
df_test = df_test.drop(columns='id',axis=1)

In [ ]:
df_test.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,3839,D-penicillamine,19724,F,N,Y,N,N,1.2,546.0,3.37,65.0,1636.0,151.90,90.0,430.0,10.6,2.0
1,2468,D-penicillamine,14975,F,N,N,N,N,1.1,660.0,4.22,94.0,1257.0,151.90,155.0,227.0,10.0,2.0
2,51,Placebo,13149,F,N,Y,N,Y,2.0,151.0,2.96,46.0,961.0,69.75,101.0,213.0,13.0,4.0
3,2330,D-penicillamine,20510,F,N,N,N,N,0.6,293.0,3.85,40.0,554.0,125.55,56.0,270.0,10.6,2.0
4,1615,D-penicillamine,21904,F,N,Y,N,N,1.4,277.0,2.97,121.0,1110.0,125.00,126.0,221.0,9.8,1.0


In [ ]:
df['Status'] = df['Status'].astype('category')
df['Status'] = df['Status'].cat.codes
df['Status'].value_counts()

0    4965
2    2665
1     275
Name: Status, dtype: int64

In [ ]:
df.columns

Index(['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders',
       'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos',
       'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage', 'Status'],
      dtype='object')

In [ ]:
df.shape, df_test.shape

((7905, 19), (5271, 18))

In [ ]:
X = df.drop(['Status'], axis=1)
Y = df['Status']

In [ ]:
X.shape

(7905, 18)

In [ ]:
X.dtypes

N_Days             int64
Drug              object
Age                int64
Sex               object
Ascites           object
Hepatomegaly      object
Spiders           object
Edema             object
Bilirubin        float64
Cholesterol      float64
Albumin          float64
Copper           float64
Alk_Phos         float64
SGOT             float64
Tryglicerides    float64
Platelets        float64
Prothrombin      float64
Stage            float64
dtype: object

In [ ]:
df_test.dtypes

N_Days             int64
Drug              object
Age                int64
Sex               object
Ascites           object
Hepatomegaly      object
Spiders           object
Edema             object
Bilirubin        float64
Cholesterol      float64
Albumin          float64
Copper           float64
Alk_Phos         float64
SGOT             float64
Tryglicerides    float64
Platelets        float64
Prothrombin      float64
Stage            float64
dtype: object

In [ ]:
X['Cholesterol'] = X['Cholesterol'].astype('float64')
X['Copper'] = X['Copper'].astype('float64')
X['Tryglicerides'] = X['Tryglicerides'].astype('float64')
X['Platelets'] = X['Platelets'].astype('float64')
X['Stage'] = X['Stage'].astype('float64')


In [ ]:
cat_features = np.where(X.dtypes != np.float)[0]

<ipython-input-66-2e6f94f2c5a4>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cat_features = np.where(X.dtypes != np.float)[0]


In [ ]:
cat_features

array([0, 1, 2, 3, 4, 5, 6, 7])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
train_pool = Pool(X_train, Y_train,cat_features=cat_features)
test_pool = Pool(X_test, Y_test,cat_features=cat_features)

In [ ]:
test_pool

In [ ]:
X_train.shape,Y_train.shape

((5928, 18), (5928,))

In [ ]:
X_test.shape, Y_test.shape

((1977, 18), (1977,))

In [ ]:
clf = CatBoostClassifier(
    eval_metric='HammingLoss',
    iterations=4000
)
clf.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=50)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.065887
0:	learn: 0.2201417	test: 0.2245827	best: 0.2245827 (0)	total: 26.1ms	remaining: 1m 44s
50:	learn: 0.1769568	test: 0.1831057	best: 0.1831057 (50)	total: 1.03s	remaining: 1m 19s
100:	learn: 0.1673414	test: 0.1780475	best: 0.1780475 (90)	total: 2.02s	remaining: 1m 18s
150:	learn: 0.1580634	test: 0.1770359	best: 0.1770359 (130)	total: 3.03s	remaining: 1m 17s
200:	learn: 0.1474359	test: 0.1760243	best: 0.1755185 (170)	total: 4.9s	remaining: 1m 32s
250:	learn: 0.1400135	test: 0.1740010	best: 0.1740010 (230)	total: 6.92s	remaining: 1m 43s
300:	learn: 0.1337719	test: 0.1729894	best: 0.1729894 (290)	total: 8.31s	remaining: 1m 42s
350:	learn: 0.1273617	test: 0.1745068	best: 0.1729894 (290)	total: 9.31s	remaining: 1m 36s
400:	learn: 0.1209514	test: 0.1740010	best: 0.1724836 (380)	total: 10.3s	remaining: 1m 32s
450:	learn: 0.1152159	test: 0.1775417	best: 0.1724836 (380)	total: 11.4s	remaining: 1m 29s
500:	learn: 0.1101552	test: 0.1765301	best: 0.1724836 (380)	total: 

In [ ]:
y_preds = clf.predict_proba(df_test)

In [ ]:
y_preds.shape

(5271, 3)

In [ ]:
y_preds

array([[7.00676495e-01, 3.99786845e-03, 2.95325637e-01],
       [5.37442339e-01, 3.65793990e-01, 9.67636718e-02],
       [1.71888322e-03, 1.01583674e-04, 9.98179533e-01],
       ...,
       [9.66125312e-01, 1.09936252e-03, 3.27753251e-02],
       [9.99589522e-01, 3.77583259e-05, 3.72719600e-04],
       [6.36836424e-01, 4.48469232e-03, 3.58678883e-01]])

In [ ]:
# prompt: make  a new csv file as res, and in it make three columns, id, Status_C, Status_CL, Status_D, id will start from 7905 and continue to go on, take values from y_preds, dont use df_

res = pd.DataFrame({
    'id': np.arange(7905, 7905+len(y_preds)),
    'Status_C': y_preds[:, 0],
    'Status_CL': y_preds[:, 1],
    'Status_D': y_preds[:, 2]
})
res
res.to_csv('res.csv', index=False)


In [ ]:
CatBoostClassifier?

In [ ]:
!pip install optuna

In [ ]:
import catboost as cb
from sklearn.metrics import log_loss
import optuna


In [ ]:
def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        # "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }
    model = CatBoostClassifier(**params, silent=True)
    model.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=50)
    predictions = model.predict_proba(test_pool)
    logloss = log_loss(Y_test, predictions)
    return logloss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2023-12-21 13:56:26,081] A new study created in memory with name: no-name-e26368f1-78ab-49fc-b001-3caed6eb3eea


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.0954517	test: 1.0954985	best: 1.0954985 (0)	total: 22.6ms	remaining: 22.6s
50:	learn: 0.9620241	test: 0.9640056	best: 0.9640056 (50)	total: 952ms	remaining: 17.7s
100:	learn: 0.8659014	test: 0.8697407	best: 0.8697407 (100)	total: 2.15s	remaining: 19.2s
150:	learn: 0.7933088	test: 0.7988345	best: 0.7988345 (150)	total: 3.09s	remaining: 17.4s
200:	learn: 0.7375947	test: 0.7444664	best: 0.7444664 (200)	total: 4.14s	remaining: 16.5s
250:	learn: 0.6928640	test: 0.7009663	best: 0.7009663 (250)	total: 5.83s	remaining: 17.4s
300:	learn: 0.6536330	test: 0.6605734	best: 0.6605734 (300)	total: 7.69s	remaining: 17.9s
350:	learn: 0.6220974	test: 0.6283860	best: 0.6283860 (350)	total: 9.6s	remaining: 17.7s
400:	learn: 0.5964701	test: 0.6025276	best: 0.6025276 (400)	total: 11.7s	remaining: 17.5s
450:	learn: 0.5755187	test: 0.5812932	best: 0.5812932 (450)	total: 13.8s	remaining: 16.8s
500:	learn: 0.5581069	test: 0.5637743	best: 0.5637743 (500)	total: 15.6s	remaining: 15.6s
550:	learn: 0.54

[I 2023-12-21 13:56:57,680] Trial 0 finished with value: 0.49027148372754586 and parameters: {'learning_rate': 0.0031648390467043036, 'depth': 6, 'colsample_bylevel': 0.7832730960315538, 'min_data_in_leaf': 91}. Best is trial 0 with value: 0.49027148372754586.


999:	learn: 0.4787606	test: 0.4902715	best: 0.4902715 (999)	total: 31.3s	remaining: 0us

bestTest = 0.4902714837
bestIteration = 999



...

999:	learn: 0.3156021	test: 0.4380862	best: 0.4380247 (980)	total: 12.3s	remaining: 0us

bestTest = 0.4380246629
bestIteration = 980

Shrink model to first 981 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.0698066	test: 1.0700697	best: 1.0700697 (0)	total: 19.7ms	remaining: 19.7s
50:	learn: 0.5669447	test: 0.5778503	best: 0.5778503 (50)	total: 890ms	remaining: 16.6s
100:	learn: 0.4754815	test: 0.4942892	best: 0.4942892 (100)	total: 1.59s	remaining: 14.1s
150:	learn: 0.4441254	test: 0.4733216	best: 0.4733216 (150)	total: 2.34s	remaining: 13.1s
200:	learn: 0.4238423	test: 0.4633818	best: 0.4633818 (200)	total: 3.08s	remaining: 12.2s
250:	learn: 0.4087047	test: 0.4576961	best: 0.4576961 (250)	total: 3.8s	remaining: 11.4s
300:	learn: 0.3947807	test: 0.4531923	best: 0.4531923 (300)	total: 4.47s	remaining: 10.4s
350:	learn: 0.3821705	test: 0.4509452	best: 0.4509452 (350)	total: 5.19s	remaining: 9.6s
400:	learn: 0.3699960	test: 0.4482621	best: 0.4482621 (400)	total: 5.91s	remaining: 8.82s
450:	learn: 0.3584958	test: 0.4458078	best: 0.4458078 (450)	total: 6.63s	remaining: 8.07s
500:	learn: 0.3484322	test: 0.4444769	best: 0.4444769 (500)	total: 7.38s	remaining: 7.35s
550:	learn: 0.338

[I 2023-12-21 14:03:34,976] Trial 29 finished with value: 0.4392465490275352 and parameters: {'learning_rate': 0.02890332510441484, 'depth': 7, 'colsample_bylevel': 0.2739515174757149, 'min_data_in_leaf': 83}. Best is trial 23 with value: 0.4361331198496779.


999:	learn: 0.2696790	test: 0.4393197	best: 0.4392465 (970)	total: 16.7s	remaining: 0us

bestTest = 0.439246549
bestIteration = 970

Shrink model to first 971 iterations.


In [ ]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

Best hyperparameters: {'learning_rate': 0.055346003234539996, 'depth': 5, 'colsample_bylevel': 0.20950333072274252, 'min_data_in_leaf': 1}
Best RMSE: 0.4361331198496779


In [ ]:
best_params_list = study.best_params
final_model0 = CatBoostClassifier(iterations=1000 , learning_rate = 0.055346003234539996, depth = 5, colsample_bylevel= 0.20950333072274252, min_data_in_leaf= 1)


In [ ]:
final_model0.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=50)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.0495117	test: 1.0500908	best: 1.0500908 (0)	total: 8.14ms	remaining: 8.13s
50:	learn: 0.4981893	test: 0.5022356	best: 0.5022356 (50)	total: 371ms	remaining: 6.9s
100:	learn: 0.4592886	test: 0.4682259	best: 0.4682259 (100)	total: 721ms	remaining: 6.42s
150:	learn: 0.4414082	test: 0.4584590	best: 0.4584590 (150)	total: 1.1s	remaining: 6.2s
200:	learn: 0.4256808	test: 0.4521682	best: 0.4521682 (200)	total: 1.47s	remaining: 5.86s
250:	learn: 0.4130418	test: 0.4484965	best: 0.4484965 (250)	total: 1.83s	remaining: 5.46s
300:	learn: 0.4021452	test: 0.4453641	best: 0.4453641 (300)	total: 2.17s	remaining: 5.05s
350:	learn: 0.3920433	test: 0.4434107	best: 0.4434107 (350)	total: 2.53s	remaining: 4.67s
400:	learn: 0.3818240	test: 0.4422621	best: 0.4422621 (400)	total: 2.86s	remaining: 4.27s
450:	learn: 0.3733593	test: 0.4409960	best: 0.4409566 (440)	total: 3.2s	remaining: 3.9s
500:	learn: 0.3645120	test: 0.4401521	best: 0.4401521 (500)	total: 3.56s	remaining: 3.54s
550:	learn: 0.357488

In [ ]:
preds = final_model0.predict_proba(df_test)
preds

array([[0.56799073, 0.03792587, 0.3940834 ],
       [0.43690142, 0.38724882, 0.17584976],
       [0.02393831, 0.00897293, 0.96708876],
       ...,
       [0.88405149, 0.02173035, 0.09421816],
       [0.99034282, 0.00241046, 0.00724672],
       [0.46750184, 0.02749042, 0.50500774]])

In [ ]:
res = pd.DataFrame({
    'id': np.arange(7905, 7905+len(y_preds)),
    'Status_C': preds[:, 0],
    'Status_CL': preds[:, 1],
    'Status_D': preds[:, 2]
})
res
res.to_csv('res.csv', index=False)